In [3]:
import os
# Directory to save the generated contracts
SECURE_DIR = "./contracts/secure"
INSECURE_DIR = "./contracts/insecure"

os.makedirs(SECURE_DIR, exist_ok=True)
os.makedirs(INSECURE_DIR, exist_ok=True)

# Example templates
INSECURE_TEMPLATE = """\
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract InsecureExample{index} {{
    {content}
}}
"""

SECURE_TEMPLATE = """\
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract SecureExample{index} {{
    {content}
}}
"""

# Example vulnerability patterns and fixes
VULNERABILITIES = [
    # Reentrancy
    {
        "insecure": """
        mapping(address => uint256) public balances;

        function deposit() external payable {{
            balances[msg.sender] += msg.value;
        }}

        function withdraw() external {{
            uint256 amount = balances[msg.sender];
            require(amount > 0, "No balance to withdraw");
            (bool success, ) = msg.sender.call{{value: amount}}("");
            require(success, "Transfer failed");
            balances[msg.sender] = 0; // Vulnerable: State change happens after external call
        }}
        """,
        "secure": """
        mapping(address => uint256) public balances;

        function deposit() external payable {{
            balances[msg.sender] += msg.value;
        }}

        function withdraw() external {{
            uint256 amount = balances[msg.sender];
            require(amount > 0, "No balance to withdraw");
            balances[msg.sender] = 0; // Secure: State change happens before external call
            (bool success, ) = msg.sender.call{{value: amount}}("");
            require(success, "Transfer failed");
        }}
        """
    },
    # Access Control
    {
        "insecure": """
        address public owner;

        constructor() {{
            owner = msg.sender;
        }}

        function withdrawFunds() public {{
            // No access control mechanism
            payable(msg.sender).transfer(address(this).balance);
        }}

        receive() external payable {{}}
        """,
        "secure": """
        address public owner;

        modifier onlyOwner() {{
            require(msg.sender == owner, "Not the owner");
            _;
        }}

        constructor() {{
            owner = msg.sender;
        }}

        function withdrawFunds() public onlyOwner {{
            payable(owner).transfer(address(this).balance);
        }}

        receive() external payable {{}}
        """
    },
    # Arithmetic Overflows and Underflows
    {
        "insecure": """
        uint256 public total;

        function add(uint256 value) public {{
            total += value; // Vulnerable to overflow
        }}
        """,
        "secure": """
        uint256 public total;

        function add(uint256 value) public {{
            require(value <= type(uint256).max - total, "Overflow protection");
            total += value; // Safe from overflow
        }}
        """
    }
]

# Generate files
def generate_contracts(num_contracts):
    for i in range(num_contracts):
        for j, vuln in enumerate(VULNERABILITIES):
            # Generate insecure contract
            insecure_content = INSECURE_TEMPLATE.format(index=f"{i}_{j}", content=vuln["insecure"])
            with open(f"{INSECURE_DIR}/InsecureExample{i}_{j}.sol", "w") as f:
                f.write(insecure_content)

            # Generate secure contract
            secure_content = SECURE_TEMPLATE.format(index=f"{i}_{j}", content=vuln["secure"])
            with open(f"{SECURE_DIR}/SecureExample{i}_{j}.sol", "w") as f:
                f.write(secure_content)

# Generate 50 secure and 50 insecure contracts
generate_contracts(50)
